# TrendModel Demo

In [ ]:
# Import required libraries
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple, Dict, Any
import torch.nn as nn
import torch.optim as optim

from geospatial_neural_adapter.data.generators import generate_time_synthetic_data
from geospatial_neural_adapter.data.preprocessing import prepare_all_with_scaling, denormalize_predictions
from geospatial_neural_adapter.models.trend_model import TrendModel, train_trend_model

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
plt.style.use('default')
sns.set_palette("husl")

print("✅ All imports successful!")

## 1. Data Generation with Meaningful Correlations

We'll use the improved data generator that creates features with strong correlations to targets, enabling proper scale learning.

In [ ]:
# Generate synthetic data with meaningful correlations
print("Generating correlated synthetic data...")

n_locations = 50
n_time_steps = 200
locations = np.linspace(-5, 5, n_locations)

cat_features, cont_features, targets = generate_time_synthetic_data(
    locs=locations,
    n_time_steps=n_time_steps,
    noise_std=1.0,
    eigenvalue=2.0,
    eta_rho=0.8,
    f_rho=0.6,
    global_mean=50.0,
    feature_noise_std=0.1,
    non_linear_strength=0.2,
    seed=42
)

print(f"Data shapes: {cont_features.shape}, {targets.shape}")
print(f"Original targets - Mean: {targets.mean():.2f}, Std: {targets.std():.2f}")
print(f"Original targets - Range: {targets.min():.2f} to {targets.max():.2f}")

In [ ]:
# Analyze feature-target correlations
print("Feature-Target Correlations:")
for i in range(cont_features.shape[-1]):
    corr = np.corrcoef(targets.flatten(), cont_features[:, :, i].flatten())[0, 1]
    print(f"  Feature {i}: {corr:.4f}")

# Visualize data characteristics
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Target distribution
axes[0, 0].hist(targets.flatten(), bins=30, alpha=0.7, edgecolor='black')
axes[0, 0].set_title('Target Distribution')
axes[0, 0].set_xlabel('Target Value')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Spatial pattern at first time step
axes[0, 1].plot(locations, targets[0, :], 'o-', linewidth=2, markersize=4)
axes[0, 1].set_title('Spatial Pattern at t=0')
axes[0, 1].set_xlabel('Location')
axes[0, 1].set_ylabel('Target Value')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Temporal pattern at middle location
time_steps = np.arange(len(targets))
axes[1, 0].plot(time_steps, targets[:, 25], linewidth=2)
axes[1, 0].set_title('Temporal Pattern at Location 25')
axes[1, 0].set_xlabel('Time Step')
axes[1, 0].set_ylabel('Target Value')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Feature correlations
feature_corrs = []
for i in range(cont_features.shape[-1]):
    corr = np.corrcoef(targets.flatten(), cont_features[:, :, i].flatten())[0, 1]
    feature_corrs.append(corr)

axes[1, 1].bar(range(len(feature_corrs)), feature_corrs, alpha=0.7, edgecolor='black')
axes[1, 1].set_title('Feature-Target Correlations')
axes[1, 1].set_xlabel('Feature Index')
axes[1, 1].set_ylabel('Correlation')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Enhanced Preprocessing with Automatic Scaling

We'll use the improved preprocessing pipeline that handles normalization and denormalization automatically.

In [ ]:
# Prepare datasets with automatic scaling
print("Preparing datasets with automatic scaling...")

train_dataset, val_dataset, test_dataset, preprocessor = prepare_all_with_scaling(
    cat_features=cat_features,
    cont_features=cont_features,
    targets=targets,
    train_ratio=0.7,
    val_ratio=0.15,
    feature_scaler_type="standard",
    target_scaler_type="standard",
    fit_on_train_only=True
)

train_cat, train_cont, train_targets = train_dataset.tensors
val_cat, val_cont, val_targets = val_dataset.tensors
test_cat, test_cont, test_targets = test_dataset.tensors

print(f"Dataset sizes: {len(train_dataset)}, {len(val_dataset)}, {len(test_dataset)}")

# Print scaler information
scaler_info = preprocessor.get_scaler_info()
print(f"Target scaler - Mean: {scaler_info['target_mean'][0]:.2f}, Std: {scaler_info['target_scale'][0]:.2f}")

In [ ]:
# Visualize the effect of standardization
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Original vs standardized distributions
axes[0].hist(targets.flatten(), bins=30, alpha=0.7, label='Original', density=True)
axes[0].hist(train_targets.numpy().flatten(), bins=30, alpha=0.7, label='Standardized', density=True)
axes[0].set_title('Target Distribution Comparison')
axes[0].set_xlabel('Value')
axes[0].set_ylabel('Density')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Original vs standardized spatial patterns
val_targets_orig = denormalize_predictions(val_targets.numpy(), preprocessor)
axes[1].plot(locations, val_targets_orig[0], 'o-', label='Original', alpha=0.7, linewidth=2)
axes[1].plot(locations, val_targets[0].numpy(), 's-', label='Standardized', alpha=0.7, linewidth=2)
axes[1].set_title('Spatial Pattern Comparison')
axes[1].set_xlabel('Location')
axes[1].set_ylabel('Value')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. TrendModel Architecture


In [ ]:
# Initialize the model
num_continuous_features = cont_features.shape[-1]
hidden_layer_sizes = [128, 64, 32]

model = TrendModel(
    n_locations=n_locations,
    num_continuous_features=num_continuous_features,
    hidden_layer_sizes=hidden_layer_sizes,
    dropout_rate=0.1,
)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Model initialized on device: {device}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

# Model summary
print("\nModel Architecture:")
print(model)

## 4. Training Function with Modern Optimization

We'll implement a training function with modern optimization techniques for better convergence.

In [ ]:
# Train the model
print("Training Improved TrendModel...")
trained_model, train_losses, val_losses = train_trend_model(
    model=model,
    train_cont=train_cont,
    train_targets=train_targets,
    val_cont=val_cont,
    val_targets=val_targets,
    num_epochs=100,
    learning_rate=1e-3,
    device=device,
    patience=20
)

print(f"Training completed! Final train loss: {train_losses[-1]:.4f}, Final val loss: {val_losses[-1]:.4f}")

In [ ]:
# Visualize training progress
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Training and validation loss
axes[0].plot(train_losses, 'b-', linewidth=2, label='Training Loss')
axes[0].plot(val_losses, 'r-', linewidth=2, label='Validation Loss')
axes[0].set_title('Training and Validation Loss Over Epochs')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('MSE Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_yscale('log')

# Loss difference (overfitting check)
loss_diff = np.array(train_losses) - np.array(val_losses)
axes[1].plot(loss_diff, 'g-', linewidth=2)
axes[1].axhline(y=0, color='k', linestyle='--', alpha=0.5)
axes[1].set_title('Training - Validation Loss Difference')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss Difference')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Model Evaluation with Denormalization

We'll evaluate the model and demonstrate proper denormalization of predictions.

In [ ]:
def evaluate_model_with_denorm(
    model: nn.Module, 
    cont_data: torch.Tensor, 
    targets: torch.Tensor, 
    preprocessor,
    device: str
) -> Tuple[np.ndarray, np.ndarray, Dict[str, float]]:
    """
    Evaluate model and return both standardized and denormalized predictions and metrics.
    """
    model.eval()
    with torch.no_grad():
        cont_data = cont_data.to(device)
        targets = targets.to(device)
        
        # Get standardized predictions
        preds_std = model(cont_data)
        
        # Calculate metrics on standardized scale
        mse_std = torch.nn.functional.mse_loss(preds_std, targets)
        mae_std = torch.nn.functional.l1_loss(preds_std, targets)
        
        # R-squared calculation on standardized scale
        ss_res = torch.sum((targets - preds_std) ** 2)
        ss_tot = torch.sum((targets - targets.mean()) ** 2)
        r2_std = 1 - (ss_res / ss_tot)
        
        # Denormalize predictions using the preprocessor
        preds_denorm = denormalize_predictions(preds_std.cpu().numpy(), preprocessor)
        targets_denorm = denormalize_predictions(targets.cpu().numpy(), preprocessor)
        
        # Calculate metrics on original scale
        mse_denorm = np.mean((targets_denorm - preds_denorm) ** 2)
        mae_denorm = np.mean(np.abs(targets_denorm - preds_denorm))
        
        # R-squared on original scale
        ss_res_denorm = np.sum((targets_denorm - preds_denorm) ** 2)
        ss_tot_denorm = np.sum((targets_denorm - targets_denorm.mean()) ** 2)
        r2_denorm = 1 - (ss_res_denorm / ss_tot_denorm)
        
    return preds_std.cpu().numpy(), preds_denorm, {
        'MSE_std': mse_std.item(),
        'MAE_std': mae_std.item(),
        'R2_std': r2_std.item(),
        'MSE_denorm': mse_denorm,
        'MAE_denorm': mae_denorm,
        'R2_denorm': r2_denorm
    }

# Evaluate model on validation and test sets
print("Evaluating model performance...")

val_preds_std, val_preds_denorm, val_metrics = evaluate_model_with_denorm(
    trained_model, val_cont, val_targets, preprocessor, device
)

test_preds_std, test_preds_denorm, test_metrics = evaluate_model_with_denorm(
    trained_model, test_cont, test_targets, preprocessor, device
)

print("Validation Metrics:")
print(f"  Standardized - MSE: {val_metrics['MSE_std']:.4f}, R²: {val_metrics['R2_std']:.4f}")
print(f"  Denormalized - MSE: {val_metrics['MSE_denorm']:.4f}, R²: {val_metrics['R2_denorm']:.4f}")

print("\nTest Metrics:")
print(f"  Standardized - MSE: {test_metrics['MSE_std']:.4f}, R²: {test_metrics['R2_std']:.4f}")
print(f"  Denormalized - MSE: {test_metrics['MSE_denorm']:.4f}, R²: {test_metrics['R2_denorm']:.4f}")

## 6. Comprehensive Results Visualization

Let's create comprehensive visualizations to demonstrate the scale detection improvements.

In [ ]:
# Create comprehensive visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Spatial pattern comparison
time_idx = 0
val_targets_orig = denormalize_predictions(val_targets.numpy(), preprocessor)
axes[0, 0].plot(locations, val_targets_orig[time_idx], 'o-', label='Actual', alpha=0.7, linewidth=2, markersize=4)
axes[0, 0].plot(locations, val_preds_denorm[time_idx], 's-', label='Predicted', alpha=0.7, linewidth=2, markersize=4)
axes[0, 0].set_title('Validation: Spatial Pattern (Original Scale)')
axes[0, 0].set_xlabel('Location')
axes[0, 0].set_ylabel('Target Value')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Predictions vs Actual scatter plot
all_targets_orig = np.concatenate([val_targets_orig.flatten(), 
                                 denormalize_predictions(test_targets.numpy(), preprocessor).flatten()])
all_preds_denorm = np.concatenate([val_preds_denorm.flatten(), test_preds_denorm.flatten()])

axes[0, 1].scatter(all_targets_orig, all_preds_denorm, alpha=0.5, s=20)
axes[0, 1].plot([all_targets_orig.min(), all_targets_orig.max()], 
                [all_targets_orig.min(), all_targets_orig.max()], 'r--', linewidth=2, label='Perfect Prediction')
axes[0, 1].set_title('Predictions vs Actual Values (Original Scale)')
axes[0, 1].set_xlabel('Actual Values')
axes[1, 0].set_ylabel('Predicted Values')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Temporal pattern
loc_idx = 25
test_targets_orig = denormalize_predictions(test_targets.numpy(), preprocessor)
time_steps = np.arange(len(test_targets_orig))
axes[1, 0].plot(time_steps, test_targets_orig[:, loc_idx], 'b-', label='Actual', linewidth=2)
axes[1, 0].plot(time_steps, test_preds_denorm[:, loc_idx], 'r-', label='Predicted', linewidth=2)
axes[1, 0].set_title(f'Temporal Pattern at Location {loc_idx} (Original Scale)')
axes[1, 0].set_xlabel('Time Step')
axes[1, 0].set_ylabel('Target Value')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Residuals analysis
residuals = all_targets_orig - all_preds_denorm
axes[1, 1].scatter(all_preds_denorm, residuals, alpha=0.5, s=20)
axes[1, 1].axhline(y=0, color='r', linestyle='--', alpha=0.7)
axes[1, 1].set_title('Residuals vs Predicted Values')
axes[1, 1].set_xlabel('Predicted Values')
axes[1, 1].set_ylabel('Residuals')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Scale Analysis and Summary

Let's analyze the scale recovery and provide a comprehensive summary of the improvements.

In [ ]:
# Scale analysis
print("=== Scale Analysis ===")
print(f"Original target range: {targets.min():.2f} to {targets.max():.2f}")
print(f"Predicted range: {all_preds_denorm.min():.2f} to {all_preds_denorm.max():.2f}")
print(f"Scale recovery: {all_preds_denorm.max() - all_preds_denorm.min():.2f} / {targets.max() - targets.min():.2f} = {(all_preds_denorm.max() - all_preds_denorm.min()) / (targets.max() - targets.min()):.2%}")

print(f"\nOriginal std: {targets.std():.2f}")
print(f"Predicted std: {all_preds_denorm.std():.2f}")
print(f"Std recovery: {all_preds_denorm.std() / targets.std():.2%}")

# Feature importance analysis
print(f"\n=== Feature Importance ===")
for i in range(cont_features.shape[-1]):
    corr = np.corrcoef(targets.flatten(), cont_features[:, :, i].flatten())[0, 1]
    print(f"Feature {i} correlation: {corr:.4f}")

# Model performance summary
print(f"\n=== Model Performance Summary ===")
print(f"Validation R²: {val_metrics['R2_denorm']:.4f}")
print(f"Test R²: {test_metrics['R2_denorm']:.4f}")
print(f"Validation MAE: {val_metrics['MAE_denorm']:.4f}")
print(f"Test MAE: {test_metrics['MAE_denorm']:.4f}")

In [ ]:
# Final comparison visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Distribution comparison
axes[0].hist(targets.flatten(), bins=30, alpha=0.7, label='Original', density=True, edgecolor='black')
axes[0].hist(all_preds_denorm, bins=30, alpha=0.7, label='Predicted', density=True, edgecolor='black')
axes[0].set_title('Distribution Comparison')
axes[0].set_xlabel('Value')
axes[0].set_ylabel('Density')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Scale recovery visualization
metrics = ['Scale Recovery', 'Std Recovery', 'R² Score']
values = [
    (all_preds_denorm.max() - all_preds_denorm.min()) / (targets.max() - targets.min()),
    all_preds_denorm.std() / targets.std(),
    test_metrics['R2_denorm']
]
values = [v * 100 for v in values]  # Convert to percentage

bars = axes[1].bar(metrics, values, alpha=0.7, edgecolor='black')
axes[1].set_title('Performance Metrics')
axes[1].set_ylabel('Percentage (%)')
axes[1].grid(True, alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars, values):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                f'{value:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()